In [5]:
SAMPLES_DIR="../samples/"
ETC_DIR=SAMPLES_DIR+"etc/"
AUDIO_DIR=SAMPLES_DIR+"audio/"
OUT_DIR=SAMPLES_DIR+"out/"
VIDEO_NAME="Untitled.mp4"
AUDIO_NAME="Untitled.wav"

In [6]:
from pydub import AudioSegment

video = AudioSegment.from_file(SAMPLES_DIR+VIDEO_NAME, format="mp4")

In [7]:
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)

In [8]:
audio.export(AUDIO_DIR+AUDIO_NAME, format="wav")

<_io.BufferedRandom name='../samples/audio/Untitled.wav'>

In [9]:
import whisper
model = whisper.load_model("base")

In [10]:
output = model.transcribe(AUDIO_DIR+AUDIO_NAME)
print(output)

/opt/miniconda3/envs/sportlight/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': " the champion. And there's money. Still has a big broad smile on his face. Oh, that's come back strongly. Challenge for the Empire. Close thing. Close shape. I've got him going to be whether it's out or not. That is a superb delivery. And the other balls of shine. Much swing. This seems to straighten up nicely. Beautiful scene. Shaving back in. Gambier. Maybe a little bit high. Oh, hitting the top of leg stump. And that's that like a footwork we've talked about with Gambier. Court on the crease. It could be a tester for the right hander. His wicket taking delivery is the out swinger. No slip in place. I'm just a little surprised. This is when you want about 13 fields. More host. And he's coming in. I think Vicki Pontings got a microphone straight to you, Remise. But he's got his out swinger going. Very, very dangerous. High 130 spin-helf in house. Another tester. 34. Got him going to be on strike. Brett Lee is running in. It's gone. It's gone all the way. Difficult one for Ad

In [11]:
positive = list()

add = ["Straight", "biggie", "Cover", "OnDrive", "Square", "Forward", "stadium", "Defence", "Sweep", "Reverse",
           "FrontFoot ", "LegGlance ", "BackFoot", "SquareCut", "Pull ", "Shot", "Hook", "Uppercut", "Cut", "Helicopter ", "SwitchHit",
           "Dilscoop", "class", "bounce", "Upper", "Uppish", "Scoop ", "Inside", "Out", "Shots", "Bouncer", "Outswinger", "Inswinger",
           "ReverseSwing", "played", "LegCutter", "OffCutter", "Yorker", "Slower", "Spin", "LegBreak ", "OffBreak", "Googly ",
           "Doosra", "Topspin ", "CarromBall", "Slider", "ArmBall", "Infield", "InnerRing", "Outfield", "Catching", "Wicketkeeper",
           "Slip", "Gully", "LegSlip", "LegGully", "Sillypoint", "Sillymidoff", "Shortleg", "Sillymidon", "InnerRing", "Point", "BackwardPoint",
           "MidOff", "Cover", "MidOn", "SquareLeg", "Backward ", "SquareLeg", "MidWicket", "FineLeg", "Outfield", "ThirdMan",
           "DeepPoint", "BackwardPoint", "ExtraCover", "LongOff", "FineLeg", "LongLeg", "LongOn", "Deep", "Cover", "played", "account"
           "cricket", "hard", "sides", "man", "finishes", "one", "crucial", "Captain", "shot", "six", "four", "boundary", "line", "drive",
           "celebrate", "placement", "beauty", "fifty", "century", "perfect", "magnifcient", "world", "cup", "batting", "fielding", "bowling",
           "catch", "caught", "out", "stumped", "one", "bowled", "night", "final", "room", "taken", "edged", "wicket", "review", "DRS", "cuts", "out", "short"]

In [13]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
from nltk.corpus import wordnet

for i in add:
        for synset in wordnet.synsets(i):
            for lemma in synset.lemmas():
                positive.append(lemma.name())

In [15]:
strings = ' '.join(positive)

In [16]:
data = output['segments']

In [17]:
text = []
start_time = []
end_time = []

for i in data:
    text.append(i['text'])
    start_time.append(i['start'])
    end_time.append(i['end'])

In [18]:
import pandas as pd

def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return (df)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

score = []
for i in text:
    d = [i, strings]
    Tfidf_vect = TfidfVectorizer()
    vector_matrix = Tfidf_vect.fit_transform(d)
    tokens = Tfidf_vect.get_feature_names_out()
    create_dataframe(vector_matrix.toarray(), tokens)
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    result = create_dataframe(
        cosine_similarity_matrix, ['Phrase', 'Strings'])
    score.append(result['Phrase'].values[1])

In [20]:
stack_first = []
stack_last = []

for i in range(len(score)):
    if (score[i] >= 0.01000000000000):
        stack_first.append(start_time[i])
        stack_last.append(end_time[i])

In [21]:
# Sort both lists based on one list
# stack_first, stack_last = zip(*sorted(zip(stack_first, stack_last)))
result = sorted(zip(stack_first, stack_last))

In [26]:
pd.DataFrame(data=result, columns=["Start time", "End time"])

,Start time,End time
0,14.00,17.00
1,48.00,51.00
2,51.00,52.00
3,62.00,64.00
4,65.00,68.00
...,...,...
91,1750.38,1751.38
92,1751.38,1754.38
93,1773.38,1776.38
94,1778.38,1780.38


In [22]:
result

[(14.0, 17.0),
 (48.0, 51.0),
 (51.0, 52.0),
 (62.0, 64.0),
 (65.0, 68.0),
 (69.0, 73.0),
 (84.0, 87.0),
 (101.0, 102.0),
 (104.0, 106.0),
 (166.0, 170.0),
 (175.0, 177.0),
 (206.0, 209.0),
 (217.0, 220.0),
 (242.0, 244.0),
 (254.0, 256.0),
 (294.0, 299.0),
 (305.0, 307.0),
 (307.0, 313.0),
 (332.0, 335.0),
 (362.0, 365.0),
 (380.0, 383.0),
 (394.0, 397.0),
 (451.0, 453.0),
 (456.0, 459.0),
 (473.0, 475.0),
 (475.0, 476.0),
 (510.0, 512.0),
 (525.0, 528.0),
 (528.0, 530.0),
 (538.0, 540.0),
 (540.0, 542.0),
 (542.0, 545.0),
 (545.0, 549.0),
 (553.0, 558.0),
 (558.0, 561.0),
 (567.0, 571.0),
 (571.0, 573.0),
 (573.0, 576.0),
 (576.0, 581.0),
 (602.0, 604.0),
 (634.0, 637.0),
 (637.0, 639.0),
 (664.0, 667.0),
 (687.0, 690.0),
 (731.0, 734.0),
 (756.0, 758.0),
 (771.0, 775.0),
 (790.0, 793.0),
 (800.0, 802.0),
 (827.0, 829.0),
 (864.38, 867.38),
 (901.38, 903.38),
 (924.38, 929.38),
 (951.38, 954.38),
 (972.38, 976.38),
 (1041.38, 1043.38),
 (1078.38, 1080.38),
 (1095.38, 1097.38),
 (1101

In [23]:
res = [[result[0][0] - 4 if result[0][0] - 4 > 0 else 0, result[0][1] + 3]]
for i in range(1, len(result)):
    if res[-1][1] + 3 >= result[i][0] - 4:
        if res[-1][1] + 3 < result[i][1] + 3:
            res[-1][1] = result[i][1] + 3
    else:
        res.append([result[i][0] - 4 if result[i][0] - 4 > 0 else 0, result[i][1] + 3])

In [24]:
res

[[10.0, 20.0],
 [44.0, 54.0],
 [58.0, 76.0],
 [80.0, 90.0],
 [97.0, 109.0],
 [162.0, 180.0],
 [202.0, 223.0],
 [238.0, 259.0],
 [290.0, 316.0],
 [328.0, 338.0],
 [358.0, 368.0],
 [376.0, 386.0],
 [390.0, 400.0],
 [447.0, 462.0],
 [469.0, 478.0],
 [506.0, 515.0],
 [521.0, 584.0],
 [598.0, 607.0],
 [630.0, 640.0],
 [660.0, 670.0],
 [683.0, 693.0],
 [727.0, 737.0],
 [752.0, 761.0],
 [767.0, 778.0],
 [786.0, 805.0],
 [823.0, 832.0],
 [860.38, 870.38],
 [897.38, 906.38],
 [920.38, 932.38],
 [947.38, 957.38],
 [968.38, 979.38],
 [1037.38, 1046.38],
 [1074.38, 1083.38],
 [1091.38, 1115.38],
 [1171.38, 1193.38],
 [1207.38, 1219.38],
 [1248.38, 1266.38],
 [1270.38, 1287.38],
 [1295.38, 1317.38],
 [1329.38, 1339.38],
 [1376.38, 1405.38],
 [1411.38, 1422.38],
 [1449.38, 1462.38],
 [1473.38, 1488.38],
 [1523.38, 1534.38],
 [1548.38, 1560.38],
 [1571.38, 1582.38],
 [1590.38, 1610.38],
 [1618.38, 1627.38],
 [1639.38, 1658.38],
 [1721.38, 1729.38],
 [1746.38, 1754.38],
 [1769.38, 1783.38],
 [1804.38,

In [28]:
dff = pd.DataFrame(data=res, columns=["Start time", "End time"])

In [30]:
dff

,Start time,End time
0,10.00,20.00
1,44.00,54.00
2,58.00,76.00
3,80.00,90.00
4,97.00,109.00
5,162.00,180.00
6,202.00,223.00
7,238.00,259.00
8,290.00,316.00
9,328.00,338.00


In [29]:
dff.head(5)

,Start time,End time
0,10.0,20.0
1,44.0,54.0
2,58.0,76.0
3,80.0,90.0
4,97.0,109.0


In [25]:
#Create temporary folder for storing subclips generated. This folder will be deleted later after highlights are generated. 
import os, shutil
sub_folder=os.path.join(ETC_DIR,"Subclips")
if os.path.exists(sub_folder):
	shutil.rmtree(sub_folder)
	path=os.mkdir(sub_folder)
else:
	path=os.mkdir(sub_folder)
#print(sub_folder,type(sub_folder))

In [ ]:
#Extract moments from videos to be added in highlight
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, concatenate_videoclips
for i in range(len(res)):
	filename="highlight" + str(i+1) + ".mp4"
	ffmpeg_extract_subclip(SAMPLES_DIR+VIDEO_NAME,res[i][0],res[i][1],targetname=sub_folder+"/"+filename) #Enter your sports video clip name here.

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [ ]:
files=os.listdir(sub_folder)
files=[sub_folder+"/highlight" + str(i+1) + ".mp4" for i in range(len(res))]
#print(files)
final_clip=concatenate_videoclips([VideoFileClip(i) for i in files])
final_clip.write_videofile(OUT_DIR+VIDEO_NAME) #Enter the desired output highlights filename.
shutil.rmtree(sub_folder) #Delete the temporary file.

Moviepy - Building video ../samples/out/Untitled.mp4.
MoviePy - Writing audio in UntitledTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../samples/out/Untitled.mp4



Moviepy - Done !
Moviepy - video ready ../samples/out/Untitled.mp4
